In [ ]:
# pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area) 

# create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. 

# compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.

# choose the top 10 POIs based on the popularity (number of reviews or average rating) (Yelp, Foursquare).

In [43]:
import pandas as pd
import os

In [44]:
import requests as re
foursquare_api = os.environ.get("foursquare_api_key")

In [45]:
def places(ll):
    url = f"https://api.foursquare.com/v3/places/search?query=bubbletea&ll={ll}&radius=10000&categories=13033"
    headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api}
    response = re.request("GET", url, headers=headers)
    return pd.json_normalize(response.json(), record_path='results', errors='ignore')


In [46]:
burnaby = '49.2276,-123.0076'
surrey = '49.1913,-122.8490'
richmond =  '49.1666,-123.1336'


In [47]:
bubble_surrey = places(surrey)
bubble_burnaby = places(burnaby)
bubble_richmond = places(richmond)

In [48]:
def foursquare(lst):
    bubbletea_total = pd.DataFrame()
    for fsq_id in lst:
        url = f"https://api.foursquare.com/v3/places/{fsq_id}?fields=name%2Clocation%2Cdistance%2Cpopularity%2Cprice%2Crating"
        headers = { "Accept": "application/json", 
        "Authorization": foursquare_api}
        response = requests.request("GET", url, headers=headers)
        bubbletea_total = pd.concat([bubbletea_total, pd.json_normalize(response.json())])
    return bubbletea_total


In [49]:
bubble_final_surrey = foursquare(bubble_surrey['fsq_id'])
cols = bubble_final_surrey.columns.tolist()

cols = cols[:3] + cols[3:4]+ cols[-5:-4]

sss = bubble_final_surrey[cols].sort_values('popularity', ascending=False)

In [50]:
bubble_final_burnaby = foursquare(bubble_burnaby['fsq_id'])

cols = bubble_final_burnaby.columns.tolist()

cols = cols[:3] + cols[3:4] + cols[-6:-5]

bbb = bubble_final_burnaby[cols].sort_values('popularity', ascending=False)



In [51]:
bubble_final_richmond = foursquare(bubble_richmond['fsq_id'])
cols = bubble_final_richmond.columns.tolist()

cols = cols[:4] + cols[-6:-5] 

rrr = bubble_final_richmond[cols].sort_values('popularity', ascending=False)

In [56]:
df = [bbb, rrr, sss]
df1 = pd.concat(df).sort_values(['popularity', 'rating'], ascending=False)
fsq_bubble = df1.drop_duplicates()
fsq_bubble.head(10)

,name,popularity,rating,location.address,location.locality
0,Dragon Ball Tea House,0.992496,8.2,1007 King Edward Ave W,Vancouver
0,Milk & Sugar Cafe,0.990428,8.0,3365A Kingsway,Vancouver
0,Blossom Teas,0.971873,7.3,7198 Barnet Rd,Burnaby
0,Chatime,0.970270,8.2,5216 Kingsway,Burnaby
0,Peanut's Bubble Tea,0.965770,7.7,8260 Westminster Hwy,Richmond
0,Sharetea,0.964691,NaN,435O North Rd,Coquitlam
0,The Bubble Tea Shop,0.963741,NaN,1764 Manitoba St,Vancouver
0,Yi Fang Taiwan Fruit Tea,0.960468,NaN,10224 152 St,Surrey
0,Shiny Tea,0.957962,7.1,4000 No. 3 Rd,Richmond
0,Chatime,0.955171,6.4,10255 King George Blvd,Surrey


In [38]:
import requests
import os
import pandas as pd

yelp_api_key = os.environ.get("Yelp_api_key")

url = 'https://api.yelp.com/v3/businesses/search'

params = {
'term' : 'bubbletea',
'location' : 'Burnaby, BC',
'limit' : 30,
'sort_by' : 'rating',
'radius' : 10000
}

headers = {
'Authorization': 'Bearer %s' % yelp_api_key
}

response = requests.get(url, params=params, headers=headers)
response.json()

{'businesses': [{'id': 'goB7wK1XgJFLXJo4Kmx-pw',
   'alias': 'tealips-cafe-burnaby-4',
   'name': 'Tealips Cafe',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/_sdlmNUfddeMajJZq8hxDg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/tealips-cafe-burnaby-4?adjust_creative=htbzdrMr_kuyBct-sDeorQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=htbzdrMr_kuyBct-sDeorQ',
   'review_count': 123,
   'categories': [{'alias': 'bubbletea', 'title': 'Bubble Tea'},
    {'alias': 'sandwiches', 'title': 'Sandwiches'},
    {'alias': 'waffles', 'title': 'Waffles'}],
   'rating': 4.0,
   'coordinates': {'latitude': 49.2195414588813,
    'longitude': -122.956628519369},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '7139 Arcola Way',
    'address2': None,
    'address3': '',
    'city': 'Burnaby',
    'zip_code': 'V5E 0A6',
    'country': 'CA',
    'state': 'BC',
    'display_address': ['7139 Arcola Way', 'Burnaby, BC V5E 0A6', 'Can

In [61]:
yelp_bubble = pd.json_normalize(response.json(), record_path='businesses')
cols = yelp_bubble.columns.tolist()
cols = cols[2:3] + cols[6:7] + cols[8:9] + cols[10:11] + cols[-1:]
yelp_bubble[cols].sort_values(['review_count', 'rating'], ascending=False).head(10)

,name,review_count,rating,price,location.display_address
1,Milk & Sugar Cafe,240,4.0,$,"[101-3365 Kingsway, Vancouver, BC V5R 5K6, Can..."
3,The One Restaurant,227,3.5,$$,"[5908 Kingsway, Burnaby, BC V5J 1H2, Canada]"
22,MYST Asian Fusion,204,3.5,$$,"[2-6400 Kingsway, Burnaby, BC V5E 1C5, Canada]"
2,Bubble World,166,3.5,$$,"[4300 Kingsway, Burnaby, BC V5H 1Z8, Canada]"
9,No.1 Beef Noodle House,163,3.5,$$,"[4741 Willingdon Avenue, Burnaby, BC V5G 3H5, ..."
10,Fondway,124,3.5,$$,"[4462 Beresford Street, Burnaby, BC V5H 2Y8, C..."
0,Tealips Cafe,123,4.0,$$,"[7139 Arcola Way, Burnaby, BC V5E 0A6, Canada]"
4,Chatime,94,3.0,$,"[5216 Kingsway, Burnaby, BC V5H 2E8, Canada]"
16,OneZo Tapioca,93,3.5,$,"[4929 Kingsway, Burnaby, BC V5H 2E5, Canada]"
13,Bobacabana,85,3.5,$$,"[800 Carvarvon Street, Suite 360, New Westmins..."
